# FunOS module init analysis 

In [ ]:

# This is a notebook that generates a chart of module init time. 
# This notebook shows the module init time chart.

# Input:
#     - raw log file: load through config file

# Config file:
#    - config file: `funos_module_init_analysis_config.yml` in the same directory as this notebook


In [ ]:
# *NOTE*: run this command to clean output cell and meta data.
# $ nb-clean clean ./funos_module_init_analysis.ipynb 

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Tuple
import yaml
import json
import os
import logging

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, Markdown, Latex

In [ ]:
try:
    from funos_module_init_time import *
    from funos_module_init_time_plot import plot_module_time_chart
except ImportError:
    print("Import error")

## Load config

In [ ]:
# load config file
current_path = os.getcwd()
print("current directory is: "+current_path)

config_file = "funos_module_init_analysis_config.yml"
path_to_yaml = os.path.join(current_path, config_file)
print("path_to_config_file "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file at {} : {}'.format(path_to_yaml, e))

In [ ]:
# setup the config variables
input_file_url = config["file_names"]["input_file_url"]

## Notebook summary

In [ ]:

note_str = "This page summarizes the FunOS modules init time line.\n\n"
display(Markdown(note_str))


## Module and notif init file loading

In [ ]:
# A logger for this file
logger = logging.getLogger(__name__)


In [ ]:
# load config file
current_path = os.getcwd()
note_str = "current directory is: {}\n\n".format(current_path)
display(Markdown(note_str))

note_str = "INPUT_FILE_URL: [{}]({})\n\n".format(input_file_url, input_file_url)
display(Markdown(note_str))

fun_module_notif_init_df, result = process_module_notif_init_data(
    file_name_url=input_file_url, logger=logger, working_dir=current_path
)

note_str = "### Module init time summary\n\n"
display(Markdown(note_str))

tmp_df = pd.DataFrame(result, index=[0])
display(tmp_df.round())
del tmp_df

## Plot all modules and notif init

In [ ]:
# MAIN CALLS
# plot the events

note_str = "The following chart shows the modules and notification init durations and timelines.\n\n"
display(Markdown(note_str))

threshold_collapse = get_duration_threshold(fun_module_notif_init_df, threshold=0.01)

note_str = "**NOTE**: When the duration is less than a threshold ({} ns), the module duration is depicted as the threshold duration so that it can be seen in the chart.\n\n".format(int(threshold_collapse))

display(Markdown(note_str))

working_dir = "."
full_module_notif_chart = os.path.join(working_dir, "fun_module_notif_init_chart.png")
plot_module_time_chart(fun_module_notif_init_df, disp_granualarity_ms=1000, min_duration=threshold_collapse, debug=False,logger=logger)

## Collapsed Plot all modules and notif init

In [ ]:
# MAIN CALLS
# plot the events

note_str = "To visualize it better, modules are near each other and shorter than the threshold are collapsed together in the following chart\n\n"
display(Markdown(note_str))

fun_module_notif_init_df_collapsed, group_table_module_notif = get_collapsed_df(fun_module_notif_init_df, threshold_collapse, debug=False)

working_dir = "."

save_file_name = os.path.join(working_dir, "fun_module_notif_init_df_collapsed.png")

plot_module_time_chart(fun_module_notif_init_df_collapsed, disp_granualarity_ms=1000, debug=False, group_table=group_table_module_notif, min_duration=threshold_collapse, logger=logger)

output = print_group_table(group_table_module_notif, threshold=threshold_collapse, save_file_name=save_file_name)

note_str = "#### Collapsed group description:\n\n"
display(Markdown(note_str))

note_str = "- Notificaitons are in bold\n\n"
display(Markdown(note_str))

note_str = "- `group_number(number of modules in this group): [modules in this group]`\n\n\n\n"
display(Markdown(note_str))

# replace \n with \n\n to display in markdown
output = output.replace("\n", "\n\n")
display(Markdown(output))